                                         ENSEMBLED MODEL XGBOOST AND LGBM

In [10]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('final_test.csv')

# Define the encoding function
def linear_encode_sentiment(value):
    if value == 'Positive':
        return 1
    elif value == 'Neutral':
        return 0
    elif value == 'Negative':
        return -1
    else:
        return 0  # Default for any unexpected sentiment values

# Apply linear encoding to the 'sentiment' column
df['sentiment'] = df['sentiment'].apply(linear_encode_sentiment)

# Save the updated DataFrame back to the same file
df.to_csv('final_test.csv', index=False)

print("The sentiment encoding has been updated in the original file.")


The sentiment encoding has been updated in the original file.


In [11]:
df.describe()

,exper,incent,ticket,member,earning fm,depend,work,Graduate,language_sorted,sentiment,CV_Score_1_to_5
count,187.000000,187.000000,1.870000e+02,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.0,187.000000
mean,4.231239,8491.978610,7.173797e+05,3.368984,0.978610,2.593583,0.874332,0.695936,0.631016,0.0,3.254587
std,4.274825,5207.902374,6.916836e+05,1.370857,1.140445,1.423898,0.330337,0.379070,0.760377,0.0,0.630898
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.0,1.000000
25%,2.000000,5000.000000,1.250000e+05,3.000000,0.000000,2.000000,1.000000,0.430000,1.000000,0.0,2.923092
50%,3.166667,8000.000000,3.500000e+05,3.000000,1.000000,3.000000,1.000000,1.000000,1.000000,0.0,3.166833
75%,6.000000,15000.000000,1.200000e+06,5.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.0,3.832908
max,40.000000,15000.000000,2.000000e+06,5.000000,5.000000,7.000000,1.000000,1.000000,1.000000,0.0,4.610969


In [18]:
df.sentiment.value_counts()

sentiment
0    187
Name: count, dtype: int64

In [16]:
# Check the current TensorFlow version
!pip show tensorflow

# Upgrade TensorFlow if needed
!pip install --upgrade tensorflow


Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\Nityam\anaconda3\Lib\site-packages
Requires: tensorflow-intel
Required-by: 
  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl (375.9 MB)
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
  Rolling back uninstall of tensorflow
  Moving to c:\users\nityam\anaconda3\lib\site-packages\tensorflow-2.18.0.dist-info\
   from C:\Users\Nityam\anaconda3\Lib\site-packages\~ensorflow-2.18.0.dist-info


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Nityam\\anaconda3\\Lib\\site-packages\\tensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.



                                                 ANN MODEL

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Read the data
data = pd.read_csv('final_train.csv')

# Drop unnecessary columns
data = data.drop(columns=['CandidateID', 'detected_languages','sentiment'])

# Split data into training and testing sets (80% train, 20% test)
X = data.drop(columns=['Performance'])
y = data['Performance']

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Standardize features (important for neural networks)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build the Artificial Neural Network (ANN)
model = Sequential()

# Input layer and first hidden layer
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))

# Second hidden layer
model.add(Dense(64, activation='relu'))

# Output layer (binary classification)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test data
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"ANN Accuracy: {accuracy}")

# Load and preprocess the test data
dfg = pd.read_csv('final_test.csv')

# Define the features dynamically
train_features = X.columns.tolist()
test_features = dfg.columns.tolist()

# Ensure test features match train features (reorder if necessary)
common_features = list(set(train_features) & set(test_features))
missing_features = list(set(train_features) - set(test_features))

# Add missing features as zero-filled columns in test data
for feature in missing_features:
    dfg[feature] = 0

# Reorder test features to match train feature order
dfg = dfg[train_features]

# Impute missing values and scale test data
ll_test_imputed = imputer.transform(dfg)
ll_test_scaled = scaler.transform(ll_test_imputed)

# Predict using the trained ANN
Y_pred_ANN = (model.predict(ll_test_scaled) > 0.5).astype("int32")
print(Y_pred_ANN)


In [19]:
data.sentiment.value_counts()

sentiment
 1    704
-1     22
 0     19
Name: count, dtype: int64

In [20]:
data.shape

(745, 11)

In [21]:
df.shape

(187, 13)

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Read the data
data = pd.read_csv('final_train.csv')

# Drop unnecessary columns
data = data.drop(columns=['CandidateID', 'detected_languages','sentiment','language_sorted'])

# Split data into features (X) and target (y)
X = data.drop(columns=['Performance'])
y = data['Performance']

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Standardize features (optional for XGBoost)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train the XGBoost Classifier with parameters for slight overfitting
xgb_model = XGBClassifier(
    eval_metric='logloss',
    use_label_encoder=False,
    max_depth=15,               # Increase tree depth
    learning_rate=0.01,         # Slightly higher learning rate
    n_estimators=300,          # Increase number of estimators
    subsample=0.9,             # Use more data for each tree
    colsample_bytree=0.9,      # Use more features for each tree
    reg_alpha=0.1,             # Reduce L1 regularization
    reg_lambda=0.1             # Reduce L2 regularization
)
xgb_model.fit(X_train, y_train)

# Evaluate the model on the test data
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Accuracy: {accuracy}")

# Load and preprocess the test data
dfg = pd.read_csv('final_test.csv')

# Define the features dynamically
train_features = X.columns.tolist()
test_features = dfg.columns.tolist()

# Ensure test features match train features (reorder if necessary)
common_features = list(set(train_features) & set(test_features))
missing_features = list(set(train_features) - set(test_features))

# Add missing features as zero-filled columns in test data
for feature in missing_features:
    dfg[feature] = 0

# Reorder test features to match train feature order
dfg = dfg[train_features]

# Impute missing values and scale test data
ll_test_imputed = imputer.transform(dfg)
ll_test_scaled = scaler.transform(ll_test_imputed)

# Predict using the trained XGBoost model
Y_pred_XGB = xgb_model.predict(ll_test_scaled)
print(Y_pred_XGB)


XGBoost Accuracy: 0.610738255033557
[1 0 1 1 0 1 1 0 1 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 1 1
 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0
 0 1 0 1 0 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0
 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 1
 0 0]


In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier

# Load the training dataset
data = pd.read_csv('final_train.csv')

# Drop unnecessary columns
data = data.drop(columns=['CandidateID', 'detected_languages'])

# Split data into features and target
X = data.drop(columns=['Performance'])
y = data['Performance']

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Initialize Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)
lgbm_model = lgb.LGBMClassifier(n_estimators=100, random_state=42)

# Initialize ensemble model
ensemble_model = VotingClassifier(estimators=[
    ('rf', rf_model),
    ('xgb', xgb_model),
    ('lgbm', lgbm_model)
], voting='hard')

# Lists to store accuracies for each fold
train_accuracies = []
test_accuracies = []

# Cross-validation loop
for train_index, test_index in skf.split(X_scaled, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the ensemble model
    ensemble_model.fit(X_train, y_train)

    # Predict and evaluate on train data
    y_pred_train = ensemble_model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_pred_train)

    # Predict and evaluate on test data
    y_pred_test = ensemble_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Store accuracies
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

# Print the average accuracies over all folds
print(f"Average Train Accuracy: {np.mean(train_accuracies)}")
print(f"Average Test Accuracy: {np.mean(test_accuracies)}")

# Now, make predictions on new data using the trained ensemble model
dfg = pd.read_csv('final_test.csv')
features = ['exper', 'incent', 'ticket', 'member', 'earning fm', 'depend', 'work', 'Graduate','sentiment', 'language_sorted' ]

# Extract the relevant features for the test set
ll_test = dfg[features]

# Ensure feature alignment between training and test data
ll_test = ll_test.reindex(columns=X.columns, fill_value=0)

# Impute missing values and scale test data
ll_test_imputed = imputer.transform(ll_test)
ll_test_scaled = scaler.transform(ll_test_imputed)

# Predict using the trained ensemble model
Y_pred_ensemble = ensemble_model.predict(ll_test_scaled)
print("Predictions on Test Data:", Y_pred_ensemble)


AttributeError: 'super' object has no attribute '__sklearn_tags__'